In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Загрузка и исследование данных

In [3]:
# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

In [5]:
comments.head()

0                 haha you guys are a bunch of losers.
1    Yet call out all Muslims for the acts of a few...
2    This bitch is nuts. Who would read a book by a...
3                                     You're an idiot.
4    Who cares!? Stark trek and Star Wars fans are ...
Name: comment_text, dtype: object

In [7]:
comments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
90902 non-null  object
dtypes: object(1)
memory usage: 710.3+ KB


In [9]:
comments.shape

(90902,)

In [11]:
target.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int32

In [13]:
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: target
Non-Null Count  Dtype
--------------  -----
90902 non-null  int32
dtypes: int32(1)
memory usage: 355.2 KB


In [15]:
target.shape

(90902,)

В датасетах target и comments отсутствуют пропуски, размерность одинаковая.

#### Задание 1.

**Теперь разделим наши данные на train и test. Пусть в тест у нас пойдет 30% данных. Для этого можете использовать библиотеку train_test_split из sklearn.**

In [17]:
X_train, X_test, y_train, y_test = train_test_split(comments,
                                                    target,
                                                    test_size=0.3,
                                                    random_state=42)

print(f'X train shape: {X_train.shape}, y train shape: {y_train.shape}')
print(f'X test shape: {X_test.shape}, y test shape: {y_test.shape}')
print("\nX train:", X_train)
print("\ny train:", y_train)
print("\nX test:", X_test)
print("\ny test:", y_test)

X train shape: (63631,), y train shape: (63631,)
X test shape: (27271,), y test shape: (27271,)

X train: 12294                                    Muslim terrorist.
57506    It's ironic that these are the same groups tha...
56118    Star Wars has a wow factor that Star Trek does...
28624    The settlement is 100% appropriate.\nEnding th...
63482    Where did it say to cover your cough with your...
                               ...                        
6265     You fit perfectly in Clinton's libdem basket o...
54886    I'll bet those independent contractors have no...
76820    "Lower tier" people, especially the young, wer...
860      The Devil made her do it and the man too becau...
15795    A leak in an 8" pipe, while obviously not a go...
Name: comment_text, Length: 63631, dtype: object

y train: 12294    1
57506    0
56118    0
28624    1
63482    0
        ..
6265     1
54886    0
76820    0
860      1
15795    1
Name: target, Length: 63631, dtype: int32

X test: 80470    Not m

#### Задание 2.

**Преобразуйте текст, который вы поделили на train и test, в числовой формат с помощью функции CountVectorizer.**

In [19]:
# Обрабатываем текстовые данные с помощью CountVectorizer
vectorizer = CountVectorizer(max_features=500)
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.fit_transform(X_test)

In [21]:
print(f'X train shape: {X_train.shape}, X test shape: {X_test.shape}')
print(f'X_train_vector shape: {X_train_vector.shape}, X_test_vector shape: {X_test_vector.shape}')

X train shape: (63631,), X test shape: (27271,)
X_train_vector shape: (63631, 500), X_test_vector shape: (27271, 500)


#### Задание 3.

**Импортируйте из библиотеки sklearn логистическую регрессию LogisticRegression с параметром max_iter=2000. Для оценки модели возьмите метрику accuracy и посчитайте ее.**

In [23]:
model = LogisticRegression(max_iter=2000)

model.fit(X_train_vector, y_train)
y_pred = model.predict(X_test_vector)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6279


#### Задание 4.

**Чтобы мы смогли протестировать разные комментарии, которые приходят в голову, пропишите ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.**

In [25]:
def predict_toxic(comment):
    # Преобразование нового комментария в вектор признаков
    comment_vector = vectorizer.transform([comment])
    
    # Вероятность принадлежности к классу токсичных комментариев
    pred_proba = model.predict_proba(comment_vector)[0][1]
    
    # Определение классификацию (токсичен если >= 0.5 / не токсичен если < 0.5)
    if pred_proba >= 0.5:
        result = 'Комментарий токсичен'
    else:
        result = 'Комментарий не токсичен'
        
    return f"{result}. Точность оценки: {pred_proba:.4f}"

new_comment = "The previous commentator is a loser"
output = predict_toxic(new_comment)
print(output)

new_comment2 = "The previous commentator is a wonderful person"
output = predict_toxic(new_comment2)
print(output)

Комментарий токсичен. Точность оценки: 0.9360
Комментарий не токсичен. Точность оценки: 0.2910


#### Задание 5.

**Попробуйте предсказать, токсичен ли комментарий «Apples are stupid». Потом предскажите, токсичен ли комментарий «I love apples».**

In [29]:
test_comment = "Apples are stupid"
output = predict_toxic(test_comment)
print(output)

test_comment2 = "I love apples"
output = predict_toxic(test_comment2)
print(output)

Комментарий токсичен. Точность оценки: 0.9973
Комментарий не токсичен. Точность оценки: 0.4060


#### Задание 6.

**Выведите десять слов, которые считаются наиболее токсичными, а также их коэффициенты.**

In [31]:
# Cловарь признаков
feature = list(vectorizer.get_feature_names_out())
coeff = model.coef_[0]

# Список пар (коэффициент, слово) и сортировка по убыванию значений коэффициента
sort_coeff = sorted(zip(coeff, feature), reverse=True)

top10_toxic_words = [(word, coef) for coef, word in sort_coeff[:10]]

for i, (word, coef) in enumerate(top10_toxic_words):
    print(f'{i+1}. "{word}" (Коэффициент: {coef:.4f})')

1. "idiot" (Коэффициент: 6.3224)
2. "idea" (Коэффициент: 6.3061)
3. "stupid" (Коэффициент: 6.2126)
4. "stupidity" (Коэффициент: 5.5580)
5. "more" (Коэффициент: 4.9504)
6. "dumb" (Коэффициент: 4.6660)
7. "party" (Коэффициент: 4.5905)
8. "crap" (Коэффициент: 4.4008)
9. "damn" (Коэффициент: 4.0990)
10. "if" (Коэффициент: 4.0656)


#### Задание 7.

**Взгляните на самые токсичные слова из задания 6. Вызывают ли у вас удивление какие-нибудь из них? Есть ли слова, которых, кажется, не должно быть в списке?**

Слова "idea", "more", "party" и "if" несколько выбиваются из концепции и едва ли относятся к разряду "токсичные", но оставшаяся часть слов из списка топ-10 точно попадает в данную категорию.

#### Задание 8.

**Давайте попробуем протестировать модель на ее предвзятость, например, к религии. Что думаете о получившихся результатах? Есть ли у модели bias? Этичен ли он?**

In [35]:
religion_comment = "I have a christian friend"
output = predict_toxic(religion_comment)
print(output)

religion_comment2 = "I have a muslim friend"
output = predict_toxic(religion_comment2)
print(output)

religion_comment3 = "I have a white friend"
output = predict_toxic(religion_comment3)
print(output)

religion_comment4 = "I have a black friend"
output = predict_toxic(religion_comment4)
print(output)

Комментарий не токсичен. Точность оценки: 0.4057
Комментарий не токсичен. Точность оценки: 0.4057
Комментарий не токсичен. Точность оценки: 0.4220
Комментарий не токсичен. Точность оценки: 0.4183


В целом, все высказывания относятся к нормальным, и модель это определила, таким образом можно говорить о непредвзятости и этичности модели.

#### Задание 9.

**Вы заметили, что комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, поскольку онлайн-сообщество исламофобно. Какой тип предвзятости это может внести в вашу модель?**

Если в обучающих данных преобладают токсичные комментарии, касающиеся ислама, модель может научиться считать любые упоминания об исламе потенциально токсичными. Такая ситуация может привести к необоснованным ассоциациям и к ошибочной маркировке вполне нейтральных высказываний как агрессивных или враждебных. Последствием данной предвзятости становится снижение объективности и справедливости модели.

#### Задание 10.

**Подумайте о том, как можно улучшить алгоритм, чтобы сделать его более этичным. Напишите 1–2 идеи.**

1) Можно попытаться установить, есть ли повторы наиболее часто встречающихся наборов слов и словосочетаний, и, если есть, попробовать классифицировать текст не только по словам, а также по обнаруженным сочетаниям слов;
2) Можно увеличить количество нейтральных и позитивных примеров, чтобы предотвратить перекос в группах, а также попробовать удалить специфические термины.